In [1]:
import pandas as pd

# Load the datasets
keystroke_data = pd.read_csv('/work/festusedward-n/Dataset_TZ_KM/imputed_keystroke_data.csv')
mouse_data = pd.read_csv('/work/festusedward-n/Dataset_TZ_KM/mouse_modified_trimmed_clean_imputed.csv')

# 1. Check column names
print("Keystroke Data Columns:", keystroke_data.columns)
print("Mouse Data Columns:", mouse_data.columns)

# 2. Ensure both datasets have a common column (e.g., user_id or timestamp)
# Example: Check for 'user_id' column
if 'user_id' in keystroke_data.columns and 'user_id' in mouse_data.columns:
    print("'user_id' exists in both datasets.")
else:
    print("Warning: 'user_id' is missing in one or both datasets.")

# 3. Check for missing values in the common column
print("Missing values in Keystroke Data for 'user_id':", keystroke_data['user_id'].isnull().sum())
print("Missing values in Mouse Data for 'user_id':", mouse_data['user_id'].isnull().sum())

# 4. Check for duplicate rows based on the merge key ('user_id')
print("Duplicate rows in Keystroke Data for 'user_id':", keystroke_data.duplicated(subset=['user_id']).sum())
print("Duplicate rows in Mouse Data for 'user_id':", mouse_data.duplicated(subset=['user_id']).sum())

# 5. Data type consistency check
print("Keystroke Data 'user_id' type:", keystroke_data['user_id'].dtype)
print("Mouse Data 'user_id' type:", mouse_data['user_id'].dtype)

# 6. Merge the data on 'user_id' (or another common column like 'timestamp')
# If there are other common columns like 'timestamp', you can merge using those
merged_data = pd.merge(keystroke_data, mouse_data, on='user_id', how='inner')  # Adjust 'how' as needed ('inner', 'left', 'right')

# Check the result after merge
print(f"Merged Data Shape: {merged_data.shape}")
print("Merged Data (First 5 rows):", merged_data.head())

# 7. Save the merged dataset
merged_data.to_csv('/work/festusedward-n/Dataset_TZ_KM/combined_imputed_mouse_keystroke.csv', index=False)


Keystroke Data Columns: Index(['id', 'key', 'action', 'rhythm', 'dwell_time', 'flight_time',
       'up_down_time', 'session_duration', 'user_id', 'timestamp'],
      dtype='object')
Mouse Data Columns: Index(['id', 'action', 'coordinates', 'button', 'delta', 'distance', 'speed',
       'user_id', 'timestamp'],
      dtype='object')
'user_id' exists in both datasets.
Missing values in Keystroke Data for 'user_id': 0
Missing values in Mouse Data for 'user_id': 0
Duplicate rows in Keystroke Data for 'user_id': 19935
Duplicate rows in Mouse Data for 'user_id': 252335
Keystroke Data 'user_id' type: float64
Mouse Data 'user_id' type: float64
Merged Data Shape: (142044224, 18)
Merged Data (First 5 rows):      id_x key action_x  rhythm  dwell_time  flight_time  up_down_time  \
0  3067.0   1  release     0.0    0.152503     1.788202      1.788202   
1  3067.0   1  release     0.0    0.152503     1.788202      1.788202   
2  3067.0   1  release     0.0    0.152503     1.788202      1.788202   


In [2]:
# 1. Drop duplicate rows based on 'user_id'
merged_data = merged_data.drop_duplicates(subset=['user_id'])

# 2. Rename columns for clarity
merged_data.rename(columns={
    'id_x': 'keystroke_id',
    'timestamp_x': 'keystroke_timestamp',
    'id_y': 'mouse_id',
    'timestamp_y': 'mouse_timestamp',
    'action_x': 'keystroke_action',
    'action_y': 'mouse_action'
}, inplace=True)

# 3. Convert 'user_id' to integer for consistency
merged_data['user_id'] = merged_data['user_id'].astype(int)

# 4. Drop unnecessary columns (e.g., if you don't need the original timestamps)
merged_data.drop(columns=['keystroke_timestamp', 'mouse_timestamp'], inplace=True)

# 5. Check the shape and first few rows of the cleaned data
print(f"Cleaned Data Shape: {merged_data.shape}")
print("Cleaned Data (First 5 rows):", merged_data.head())

# 6. Save the cleaned and merged dataset
merged_data.to_csv('Dataset_TZ_KM/combined_mouse_keystroke_imputed_cleaned.csv', index=False)


Cleaned Data Shape: (60, 16)
Cleaned Data (First 5 rows):           keystroke_id key keystroke_action  rhythm  dwell_time  flight_time  \
0               3067.0   1          release   0.000    0.152503     1.788202   
89428           3146.0   7          release   0.000    0.111307     1.114417   
21603744        4997.0   6            press   0.100    0.100000     0.000000   
21729594        5027.0   9            press   0.101    0.101000     0.000000   
23100496        5348.0   y            press   0.093    0.093000     0.000000   

          up_down_time  session_duration  user_id  mouse_id mouse_action  \
0             1.788202         13.830218        1   64268.0         move   
89428         1.114417         13.830218        2   63980.0      pressed   
21603744      0.000000          4.274000       19   80136.0         move   
21729594      0.000000          2.280000       21   80876.0         move   
23100496      0.000000          1.385000       24   84338.0         move   

    